In [1]:
import numpy as np
from numpy.linalg import inv, pinv, matrix_rank, norm
from scipy.linalg import lu
from time import time

In [2]:
def non_zero(a):
    a = np.array(a,np.float64)
    m = a.shape[0]
    return sum(a!=0)

def choose_pivot(A):
    A = np.array(A,np.float64)
    m = A.shape[0]
    n = A.shape[1]
    r = matrix_rank(A)
    
    n_product_nz = 1e15
    pivot = [0,0]
    
    for i in range(m):
        for j in range(n):
            if(A[i,j]!=0):
                nz_row = non_zero(A[i,:]) - 1
                nz_col = non_zero(A[:,j]) - 1
                if(nz_row*nz_col < n_product_nz):
                    n_product_nz = nz_row*nz_col
                    pivot = [i,j]
    
    return pivot

In [3]:
def swap_pivot(A,pivot):
    i = pivot[0]
    j = pivot[1]
    A = np.array(A,np.float64)
    A[[0,i],:] = A[[i,0],:]
    A[:,[0,j]] = A[:,[j,0]]
    return A

In [50]:
def find_pivot_matrices(A):
    m = A.shape[0]
    n = A.shape[1]
    r = matrix_rank(A)
    
    P1 = np.eye(m)
    P2 = np.eye(n)
    
    for i in range(r):
        B = A[i:,i:]
        k,l = choose_pivot(B)   
        print(k,l)
        B = swap_pivot(B,[k,l])
        P1[[i+0,i+k],i:] = P1[[i+k,i+0],i:]
        P2[i:,[i+0,i+l]] = P2[i:,[i+l,i+0]]
        
        A[i:,i:] = B
    return P1, P2

In [67]:
A = np.array([[0,1,2,0],[2,1,0,0],[0,0,0,3],[1,0,2,0]])
A

array([[0, 1, 2, 0],
       [2, 1, 0, 0],
       [0, 0, 0, 3],
       [1, 0, 2, 0]])

In [68]:
pivot = choose_pivot(A)
pivot

[2, 3]

In [69]:
swap_pivot(A,pivot)

array([[3., 0., 0., 0.],
       [0., 1., 0., 2.],
       [0., 1., 2., 0.],
       [0., 0., 2., 1.]])

In [11]:
def get_permutation_matrix(A,axis,i,j):
    A = np.array(A,np.float64)
    m = A.shape[0]
    n = A.shape[1]
    if(axis == 0):
        P = np.eye(m)
        P[i,i] = 0
        P[j,j] = 0
        P[i,j] = 1
        P[j,i] = 1
    else:
        P = np.eye(n)
        P[i,i] = 0
        P[j,j] = 0
        P[i,j] = 1
        P[j,i] = 1
    return P

In [12]:
def elimination(A):
    A = np.array(A,np.float64)
    m = A.shape[0]
    n = A.shape[1]
    r = matrix_rank(A)
    
    factors = []
    for i in range(1,m):
        if(A[0,0]!=0):
            factor = A[i,0]/A[0,0]
        else:
            factor = 0
        A[i,:] = A[i,:] - factor*A[0,:]
        factors.append(factor)
    return A,factors

In [13]:
#Wrong and Incomplete

def LU_Markovitz(A):
    A = np.array(A,np.float64)
    m = A.shape[0]
    n = A.shape[1]
    r = matrix_rank(A)
    U = A.copy()
    L = np.eye(m,m)
    P_left = []
    P_right = []
    for i in range(min(m,n)):
        B = U[i:,i:]
        
        k,l = choose_pivot(B)
        
        B = swap_pivot(B,[k,l])
        
        P1_i = get_permutation_matrix(B,0,k,0)
        P1 = np.eye(m)
        P1[i:,i:] = P1_i
        P_left.append(P1)
        
        P2_i = get_permutation_matrix(B,1,0,l)
        P2 = np.eye(n)
        P2[i:,i:] = P2_i
        P_right.append(P2)
        
        B,factors = elimination(B)
        L[i+1:,i] = factors
        U[i:,i:] = B
        
    P1 = np.eye(m)
    P2 = np.eye(n)
    
    for i in range(len(P_left)):
        P1 = np.dot(P1,P_left[i])
    for i in range(len(P_right)):
        P2 = np.dot(P2,P_right[i])
        
    U = np.dot(P1,np.dot(A,P2)).copy()
    L = np.eye(m,m)     
    for i in range(r):
        B = U[i:,i:]
        B,factors = elimination(B)
        L[i+1:,i] = factors
        U[i:,i:] = B
        
    return P1,P2,L,U

In [14]:
A = np.array([[0,1,0],[2,0,2],[3,0,0]])
A = np.float64(A)
A

array([[0., 1., 0.],
       [2., 0., 2.],
       [3., 0., 0.]])

In [15]:
m = 40
n = 50
A = np.round(np.random.randn(m,n)/3)

In [16]:
P1,P2,L,U = LU_Markovitz(A)

In [17]:
np.round(np.dot(P1,np.dot(A,P2)) - np.dot(L,U),2)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [19]:
def non_zero_matrix(A):
    m,n = A.shape
    return np.sum(A.reshape(1,m*n)!=0)

In [20]:
#LU by Markovitz Pivoting (Sparsity preserving)
t1 = time()
P1,P2,L,U = LU_Markovitz(A)
t2 = time()

print("Markovitz Pivoting - Sparsity Analysis\n")
print("Dimensions of A = {},{}".format(m,n))
print("No of entries in A = {}".format(m*n))
print("No on non_zero entries in A = {}".format(non_zero_matrix(A)))
print("No on non_zero entries in L = {}".format(non_zero_matrix(L)))
print("No on non_zero entries in U = {}".format(non_zero_matrix(U)))
print("No on non_zero entries in L'L = {}".format(non_zero_matrix(np.dot(np.transpose(L),L))))
print("Time taken = {}".format(t2-t1))

Markovitz Pivoting - Sparsity Analysis

Dimensions of A = 40,50
No of entries in A = 2000
No on non_zero entries in A = 276
No on non_zero entries in L = 51
No on non_zero entries in U = 296
No on non_zero entries in L'L = 70
Time taken = 0.8480000495910645


In [21]:
#Scipy default Pivoting
t1 = time()
P0,L0,U0 = lu(A)
t2 = time()


print("Scipy Default Pivoting - Sparsity Analysis\n")
print("Dimensions of A = {},{}".format(m,n))
print("No of entries in A = {}".format(m*n))
print("No on non_zero entries in A = {}".format(non_zero_matrix(A)))
print("No on non_zero entries in L = {}".format(non_zero_matrix(L0)))
print("No on non_zero entries in U = {}".format(non_zero_matrix(U0)))
print("No on non_zero entries in L'L = {}".format(non_zero_matrix(np.dot(np.transpose(L0),L0))))
print("Time taken = {}".format(t2-t1))

Scipy Default Pivoting - Sparsity Analysis

Dimensions of A = 40,50
No of entries in A = 2000
No on non_zero entries in A = 276
No on non_zero entries in L = 434
No on non_zero entries in U = 787
No on non_zero entries in L'L = 1376
Time taken = 0.0009999275207519531


In [22]:
A = np.eye(m,n) + np.round(np.random.randn(m,n)/4)

In [23]:
#LU by Markovitz Pivoting (Sparsity preserving)
t1 = time()
P1,P2,L,U = LU_Markovitz(A)
t2 = time()

print("Markovitz Pivoting - Sparsity Analysis\n")
print("Dimensions of A = {},{}".format(m,n))
print("No of entries in A = {}".format(m*n))
print("No on non_zero entries in A = {}".format(non_zero_matrix(A)))
print("No on non_zero entries in L = {}".format(non_zero_matrix(L)))
print("No on non_zero entries in U = {}".format(non_zero_matrix(U)))
print("No on non_zero entries in L'L = {}".format(non_zero_matrix(np.dot(np.transpose(L),L))))
print("Time taken = {}".format(t2-t1))

Markovitz Pivoting - Sparsity Analysis

Dimensions of A = 40,50
No of entries in A = 2000
No on non_zero entries in A = 145
No on non_zero entries in L = 41
No on non_zero entries in U = 145
No on non_zero entries in L'L = 42
Time taken = 0.3749985694885254


In [24]:
#Scipy default Pivoting
t1 = time()
P0,L0,U0 = lu(A)
t2 = time()


print("Scipy Default Pivoting - Sparsity Analysis\n")
print("Dimensions of A = {},{}".format(m,n))
print("No of entries in A = {}".format(m*n))
print("No on non_zero entries in A = {}".format(non_zero_matrix(A)))
print("No on non_zero entries in L = {}".format(non_zero_matrix(L0)))
print("No on non_zero entries in U = {}".format(non_zero_matrix(U0)))
print("No on non_zero entries in L'L = {}".format(non_zero_matrix(np.dot(np.transpose(L0),L0))))
print("Time taken = {}".format(t2-t1))

Scipy Default Pivoting - Sparsity Analysis

Dimensions of A = 40,50
No of entries in A = 2000
No on non_zero entries in A = 145
No on non_zero entries in L = 192
No on non_zero entries in U = 384
No on non_zero entries in L'L = 704
Time taken = 0.0009999275207519531


In [25]:
np.round(np.dot(P1,np.dot(A,P2)) - np.dot(L,U),2)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [26]:
non_zero_matrix(np.round(np.dot(P1,np.dot(A,P2)) - np.dot(L,U),2))

0